# Дифференциальные уравнения

Представьте, что вы пишете компьютерную игру. Вам не нужно заранее рассчитывать траекторию каждого объекта. Вам нужно лишь задать **правила**, по которым он будет двигаться в каждый следующий момент времени.

Например, для летящего снаряда правило такое: "в следующую секунду твоя вертикальная скорость уменьшится под действием гравитации".

> **Дифференциальное уравнение (ДУ) — это и есть такое правило, которое описывает, как изменится состояние системы в следующий момент времени, исходя из её текущего состояния.**

*   **Состояние** — это, например, количество кроликов в популяции или положение маятника.
*   **Правило изменения (производная)** — это скорость роста популяции или скорость движения маятника.

**Решить ДУ** — значит запустить симуляцию по этим правилам и посмотреть, что получится с течением времени. В этом нам помогут компьютеры.

## Пример 1: Рост популяции (Уравнение первого порядка)

Давайте смоделируем рост популяции бактерий.

**Правило (ДУ):** Скорость роста популяции ($y'$) прямо пропорциональна текущему количеству бактерий ($y$). Чем их больше, тем быстрее они делятся.
$$
y' = k \cdot y
$$
*   `y` — количество бактерий.
*   `k` — коэффициент роста (насколько "быстро" они размножаются).

Мы не будем решать это уравнение "на бумаге". Вместо этого мы скажем компьютеру: "Начни со 100 бактерий и на каждом шаге времени увеличивай их количество в соответствии с этим правилом". Для этого в `SciPy` есть специальная функция `solve_ivp` (решить задачу с начальными значениями).

### Интерактивная симуляция

**Поиграйте со слайдерами:**
*   **`initial_population`**: Измените, сколько бактерий было в самом начале.
*   **`growth_rate`**: Увеличьте или уменьшите скорость их размножения.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider, IntSlider

plt.style.use('seaborn-v0_8-whitegrid')

In [5]:
# Определяем "правило" - правую часть нашего ДУ: y' = f(t, y)
def population_growth(t, y, k):
    # t - время (в нашей простой модели не используется, но требуется функцией)
    # y - текущее состояние (количество бактерий)
    # k - коэффициент роста
    return k * y

# Функция для интерактивной симуляции
def simulate_population(initial_population, growth_rate):
    # Временной интервал симуляции: от 0 до 5 "часов"
    t_span = [0, 5]

    # Решаем ДУ
    # solve_ivp(функция-правило, интервал, начальное состояние, доп. аргументы для правила)
    solution = solve_ivp(
        population_growth,
        t_span,
        [initial_population],
        args=(growth_rate,),
        dense_output=True # Нужно для построения гладкого графика
    )

    # Готовим данные для графика
    t_plot = np.linspace(t_span[0], t_span[1], 200)
    y_plot = solution.sol(t_plot)[0]

    # Визуализация
    plt.figure(figsize=(10, 6))
    plt.plot(t_plot, y_plot, 'b-')
    plt.title(f'Рост популяции (Экспоненциальный закон)')
    plt.xlabel('Время (часы)')
    plt.ylabel('Количество бактерий')
    plt.show()

# Создаем интерактивные слайдеры
interact(
    simulate_population,
    initial_population=IntSlider(min=10, max=500, step=10, value=100, description='Начальная популяция'),
    growth_rate=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Коэффициент роста (k)')
);

interactive(children=(IntSlider(value=100, description='Начальная популяция', max=500, min=10, step=10), Float…

## Пример 2: Колебания маятника (Уравнение второго порядка)

Теперь рассмотрим более сложную систему — маятник. Его состояние описывается двумя величинами: **углом отклонения** и **угловой скоростью**.

**Правила (система ДУ):**
1.  Скорость изменения угла ($y_0'$) — это и есть угловая скорость ($y_1$).
2.  Скорость изменения угловой скорости ($y_1'$) — это ускорение. По законам физики, оно зависит от угла отклонения (сила тяжести тянет маятник обратно к центру) и от трения (которое замедляет движение).
$$
\begin{cases}
y_0' = y_1 \\
y_1' = - \frac{g}{L} \sin(y_0) - b \cdot y_1
\end{cases}
$$
*   `y0` — угол, `y1` — угловая скорость.
*   `g` — ускорение свободного падения.
*   `L` — длина маятника.
*   `b` — коэффициент трения.

Не пугайтесь формул! Главное — мы можем снова просто "скормить" эти два правила нашему решателю `solve_ivp`.

### Интерактивная симуляция

**Поиграйте со слайдерами:**
*   **`length`**: Посмотрите, как меняется период колебаний с изменением длины маятника.
*   **`friction`**: Увеличьте трение, и вы увидите, как колебания затухают.

In [6]:
# Физические константы
g = 9.81

# Определяем систему "правил" для маятника
def pendulum_ode(t, y, L, b):
    # y[0] - это угол (theta)
    # y[1] - это угловая скорость (omega)
    theta, omega = y

    # Правило 1: d(theta)/dt = omega
    dtheta_dt = omega
    # Правило 2: d(omega)/dt = ...
    domega_dt = - (g / L) * np.sin(theta) - b * omega

    return [dtheta_dt, domega_dt]

# Функция для интерактивной симуляции
def simulate_pendulum(length, friction):
    # Временной интервал
    t_span = [0, 20]
    # Начальное состояние: отклонили на 45 градусов (pi/4) и отпустили (скорость 0)
    initial_state = [np.pi/4, 0]

    # Решаем систему ДУ
    solution = solve_ivp(
        pendulum_ode,
        t_span,
        initial_state,
        args=(length, friction),
        dense_output=True
    )

    # Готовим данные для графика
    t_plot = np.linspace(t_span[0], t_span[1], 500)
    y_plot = solution.sol(t_plot)

    # Визуализация
    plt.figure(figsize=(12, 6))
    plt.plot(t_plot, y_plot[0], 'r-', label='Угол отклонения (рад)')
    plt.plot(t_plot, y_plot[1], 'b--', label='Угловая скорость (рад/с)')
    plt.title('Колебания маятника')
    plt.xlabel('Время (с)')
    plt.ylabel('Значение')
    plt.legend()
    plt.grid(True)
    plt.show()

# Создаем интерактивные слайдеры
interact(
    simulate_pendulum,
    length=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='Длина (L)'),
    friction=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.1, description='Трение (b)')
);

interactive(children=(FloatSlider(value=1.0, description='Длина (L)', max=5.0, min=0.1), FloatSlider(value=0.1…